In [34]:
import pandas as pd

In [35]:
bat_df = pd.read_csv('../Datasets/batter_stats.csv')
pitch_df = pd.read_csv('../Datasets/pitcher_stats.csv')
lineups = pd.read_csv('../Datasets/Lineups_2021_06_17.csv')

In [36]:
# The dots in names like J.T. were inconsistent across datasets
bat_df.loc[:,'Name'] = bat_df['Name'].str.replace('.', '')
pitch_df.loc[:,'Name'] = pitch_df['Name'].str.replace('.', '')
lineups.loc[:,' player name'] = lineups[' player name'].str.replace('.', '')

In [37]:
bats_r = bat_df[bat_df['bats'] == 'R']
bats_l = bat_df[bat_df['bats'] == 'L']

In [38]:
# Making name column consistent
lineups.columns = lineups.columns.str.replace(' player name', 'Name')

In [39]:
# Individual name that was giving trouble
lineups.loc[lineups['Name'] == 'Jonathan Gray', 'Name'] = 'Jon Gray'

In [40]:
# Creates a list of teams with no repeats
# The order needs to be kept from the dataframe because 
# opposing teams are paired
team_list = []
for team in lineups['team code']:
    if team not in team_list:
        team_list.append(team)
# Creating a dictionary of every team, and the team they face
matchups = {}
for i in range(1, len(team_list), 2):
    matchups[team_list[i]] = team_list[i-1]
    matchups[team_list[i-1]] = team_list[i]

In [41]:
lineups

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position
0,BAL,6/17/2021,1,656775.0,Cedric Mullins,1,Y,CF
1,BAL,6/17/2021,1,641820.0,Trey Mancini,2,Y,1B
2,BAL,6/17/2021,1,520471.0,Freddy Galvis,3,Y,SS
3,BAL,6/17/2021,1,663624.0,Ryan Mountcastle,4,Y,DH
4,BAL,6/17/2021,1,623993.0,Anthony Santander,5,Y,RF
...,...,...,...,...,...,...,...,...
185,SEA,6/17/2021,1,664238.0,Dylan Moore,6,N,NaN
186,SEA,6/17/2021,1,608596.0,Tom Murphy,7,N,NaN
187,SEA,6/17/2021,1,641343.0,Jake Bauers,8,N,NaN
188,SEA,6/17/2021,1,666211.0,Taylor Trammell,9,N,NaN


In [42]:
batters = lineups.loc[(lineups[' batting order'] != 'SP') & (lineups[' game_number'] == 1)]

In [43]:
pitch_df.drop(456, inplace = True)

In [44]:
matchups

{'CLE': 'BAL',
 'BAL': 'CLE',
 'SF': 'ARI',
 'ARI': 'SF',
 'TOR': 'NYY',
 'NYY': 'TOR',
 'NYM': 'CHC',
 'CHC': 'NYM',
 'ATL': 'STL',
 'STL': 'ATL',
 'HOU': 'CHW',
 'CHW': 'HOU',
 'SD': 'CIN',
 'CIN': 'SD',
 'COL': 'MIL',
 'MIL': 'COL',
 'LAA': 'DET',
 'DET': 'LAA',
 'SEA': 'TB',
 'TB': 'SEA'}

In [45]:
pitch_df.loc[pitch_df['Name']== 'Chase DeJong', 'throws']

Series([], Name: throws, dtype: object)

In [46]:
opp_pitch_hand = []
opp_pitcher = []
del_indexes = []
keep_pitch = []
for i, row in batters.iterrows():
    opp_pitch = lineups.loc[(lineups['team code'] == matchups[row['team code']])\
                            & (lineups[' batting order'] == 'SP') & (lineups[' game_number'] == 1), 'Name'].values[0]
    if len(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values) > 0:
            opp_pitch_hand.append(pitch_df.sort_values(by = 'IP', ascending = False).loc[pitch_df['Name'] == opp_pitch, 'throws'].values[0])
            opp_pitcher.append(opp_pitch)
            keep_pitch.append(opp_pitch)
    else:
        del_indexes.append(i)
        
batters.drop(index = del_indexes, inplace = True)
batters['opp_pitch_hand'] = opp_pitch_hand
batters['opp_pitch'] = opp_pitcher


/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(
<ipython-input-46-9eb6f881f0bf>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batters['opp_pitch_hand'] = opp_pitch_hand
<ipython-input-46-9eb6f881f0bf>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.htm

In [47]:
del_indexes

[48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168]

In [48]:
bats = []
k_pct = []
bb_pct = []
bb_k = []
bat_avg = []
obp = []
slg = []
ops = []
iso = []
babip = []
wrc = []
wraa = []
woba = []
wrc_plus = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    if row['opp_pitch_hand'] == 'R':
        batter_df = bat_df.loc[(bat_df['pitch']== 'R')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'L']
    if row['opp_pitch_hand'] == 'L':
        batter_df = bat_df.loc[(bat_df['pitch']== 'L')&(bat_df['Name']==row['Name'])]
        # Checking for switch hitters
        if len(batter_df) == 2:
            # A switch hitter will hit using the opposite hand of the pitcher
            batter_df = batter_df.loc[batter_df['bats']== 'R']
    if len(batter_df) == 0:
        for col in batter_df.columns:
            batter_df[col] = [float('NaN')]
    bats.append(batter_df['bats'].values[0])
    k_pct.append(batter_df['K%'].values[0])
    bb_pct.append(batter_df['BB%'].values[0])
    bb_k.append(batter_df['BB/K'].values[0])
    bat_avg.append(batter_df['AVG'].values[0])
    obp.append(batter_df['OBP'].values[0])
    slg.append(batter_df['SLG'].values[0])
    ops.append(batter_df['OPS'].values[0])
    iso.append(batter_df['ISO'].values[0])
    babip.append(batter_df['BABIP'].values[0])
    wrc.append(batter_df['wRC'].values[0])
    wraa.append(batter_df['wRAA'].values[0])
    woba.append(batter_df['wOBA'].values[0])
    wrc_plus.append(batter_df['wRC+'].values[0])
batters.loc[:,'bats'] = bats
batters.loc[:,'k_pct'] = k_pct
batters.loc[:,'bb_pct'] = bb_pct
batters.loc[:,'bb_k'] = bb_k
batters.loc[:,'bat_avg'] = bat_avg
batters.loc[:,'obp'] = obp
batters.loc[:,'slg'] = slg
batters.loc[:,'ops'] = ops
batters.loc[:,'iso'] = iso
batters.loc[:,'babip'] = babip
batters.loc[:,'wrc'] = wrc
batters.loc[:,'wraa'] = wraa
batters.loc[:,'woba'] = woba
batters.loc[:,'wrc_plus'] = wrc_plus

<ipython-input-48-0499cefeeb18>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  batter_df[col] = [float('NaN')]
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

In [49]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,bat_avg,obp,slg,ops,iso,babip,wrc,wraa,woba,wrc_plus
0,BAL,6/17/2021,1,656775.0,Cedric Mullins,1,Y,CF,R,Eli Morgan,...,0.300000,0.359684,0.465217,0.824901,0.165217,0.370588,39.537043,8.661610,0.355900,128.115117
1,BAL,6/17/2021,1,641820.0,Trey Mancini,2,Y,1B,R,Eli Morgan,...,0.245902,0.318519,0.442623,0.761141,0.196721,0.290698,18.203207,2.463103,0.332209,113.695249
2,BAL,6/17/2021,1,520471.0,Freddy Galvis,3,Y,SS,R,Eli Morgan,...,0.230769,0.317797,0.432692,0.750489,0.201923,0.250000,31.028171,2.305466,0.326958,105.352593
3,BAL,6/17/2021,1,663624.0,Ryan Mountcastle,4,Y,DH,R,Eli Morgan,...,0.261905,0.306667,0.395238,0.701905,0.133333,0.350000,26.009999,-1.170625,0.305533,94.480586
4,BAL,6/17/2021,1,623993.0,Anthony Santander,5,Y,RF,R,Eli Morgan,...,0.277174,0.318182,0.565217,0.883399,0.288043,0.268966,32.296152,7.905710,0.363411,132.919810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,SEA,6/17/2021,1,641584.0,Jake Fraley,5,N,NaN,L,Rich Hill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,SEA,6/17/2021,1,664238.0,Dylan Moore,6,N,NaN,L,Rich Hill,...,0.225000,0.318681,0.450000,0.768681,0.225000,0.259259,12.335843,1.270159,0.332544,115.726898
186,SEA,6/17/2021,1,608596.0,Tom Murphy,7,N,NaN,L,Rich Hill,...,0.205128,0.279070,0.512821,0.791890,0.307692,0.227273,6.245352,0.998650,0.337148,120.708860
187,SEA,6/17/2021,1,641343.0,Jake Bauers,8,N,NaN,L,Rich Hill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
k_9 = []
bb_9 = []
k_bb = []
hr_9 = []
k_pct = []
bb_pct = []
k_bb_pct = []
bat_avg = []
whip = []
babip = []
lob_pct = []
fip = []
xfip = []
for i, row in batters.iterrows():
    # Pulling stats based on handedness of pitcher
    pitcher_df = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== row['bats'])&(pitch_df['Name']==row['opp_pitch']),:]
    if len(pitcher_df) == 0:
        for col in pitcher_df.columns:
            pitcher_df[col] = [float('NaN')]
    k_9.append(pitcher_df['K/9'].values[0])
    bb_9.append(pitcher_df['BB/9'].values[0])
    k_bb.append(pitcher_df['K/BB'].values[0])
    hr_9.append(pitcher_df['HR/9'].values[0])
    k_pct.append(pitcher_df['K%'].values[0])
    bb_pct.append(pitcher_df['BB%'].values[0])
    k_bb_pct.append(pitcher_df['K-BB%'].values[0])
    bat_avg.append(pitcher_df['AVG'].values[0])
    whip.append(pitcher_df['WHIP'].values[0])
    babip.append(pitcher_df['BABIP'].values[0])
    lob_pct.append(pitcher_df['LOB%'].values[0])
    fip.append(pitcher_df['FIP'].values[0])
    xfip.append(pitcher_df['xFIP'].values[0])
batters.loc[:,'opp_k/9'] = k_9
batters.loc[:,'opp_bb/9'] = k_9
batters.loc[:,'opp_k/bb'] = k_9
batters.loc[:,'opp_hr/9'] = k_9
batters.loc[:,'opp_k_pct'] = k_9
batters.loc[:,'opp_bb_pct'] = k_9
batters.loc[:,'opp_k_bb_pct'] = k_9
batters.loc[:,'opp_avg'] = k_9
batters.loc[:,'opp_whip'] = k_9
batters.loc[:,'opp_babip'] = k_9
batters.loc[:,'opp_lob_pct'] = k_9
batters.loc[:,'opp_fip'] = k_9
batters.loc[:,'opp_xfip'] = k_9

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [51]:
pitchers = lineups.loc[(lineups[' batting order'] == 'SP')]

In [52]:
pitchers = pitchers[pitchers['Name'].isin(keep_pitch)]

In [53]:
batters

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,opp_hr/9,opp_k_pct,opp_bb_pct,opp_k_bb_pct,opp_avg,opp_whip,opp_babip,opp_lob_pct,opp_fip,opp_xfip
0,BAL,6/17/2021,1,656775.0,Cedric Mullins,1,Y,CF,R,Eli Morgan,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,BAL,6/17/2021,1,641820.0,Trey Mancini,2,Y,1B,R,Eli Morgan,...,3.375000,3.375000,3.375000,3.375000,3.375000,3.375000,3.375000,3.375000,3.375000,3.375000
2,BAL,6/17/2021,1,520471.0,Freddy Galvis,3,Y,SS,R,Eli Morgan,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,BAL,6/17/2021,1,663624.0,Ryan Mountcastle,4,Y,DH,R,Eli Morgan,...,3.375000,3.375000,3.375000,3.375000,3.375000,3.375000,3.375000,3.375000,3.375000,3.375000
4,BAL,6/17/2021,1,623993.0,Anthony Santander,5,Y,RF,R,Eli Morgan,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,SEA,6/17/2021,1,641584.0,Jake Fraley,5,N,NaN,L,Rich Hill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
185,SEA,6/17/2021,1,664238.0,Dylan Moore,6,N,NaN,L,Rich Hill,...,8.349398,8.349398,8.349398,8.349398,8.349398,8.349398,8.349398,8.349398,8.349398,8.349398
186,SEA,6/17/2021,1,608596.0,Tom Murphy,7,N,NaN,L,Rich Hill,...,8.349398,8.349398,8.349398,8.349398,8.349398,8.349398,8.349398,8.349398,8.349398,8.349398
187,SEA,6/17/2021,1,641343.0,Jake Bauers,8,N,NaN,L,Rich Hill,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
num_L_bats = []
num_R_bats = []
for i, row in pitchers.iterrows():
    opp_lineup = batters.loc[(batters['team code'] == matchups[row['team code']]),:]
    find_batters = []
    # There are batters whos data was null due to lack of plate appearances but handedness is known
    for j, row2 in opp_lineup.iterrows():
        if row2['bats'] != 'R' and row2['bats'] != 'L':
            find_bats = bat_df.loc[bat_df['Name'] == row2['Name'],:]
            if len(find_bats) > 0:
                find_batters.append((j, find_bats['bats'].values[0]))
            # If batter handedness is unknown, right is more likely
            if len(find_bats) == 0:
                find_batters.append((j, 'R'))
    for handedness in find_batters:
        opp_lineup.loc[handedness[0], 'bats'] = handedness[1]
    try:
        num_L_bats.append(opp_lineup['bats'].value_counts()['L'])
    except:
        num_L_bats.append(0)
    try:
        num_R_bats.append(opp_lineup['bats'].value_counts()['R'])
    except:
        num_R_bats.append(0)
pitchers['num_L_bats'] = num_L_bats
pitchers['num_R_bats'] = num_R_bats
is_nl = []
for i, row in pitchers.iterrows():
    if row['num_L_bats'] + row['num_R_bats'] == 9:
        is_nl.append(0)
    else:
        is_nl.append(1)
pitchers['is_nl'] = is_nl

/Users/nperry725/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [55]:
k_9_l = []
bb_9_l = []
k_bb_l = []
hr_9_l = []
k_pct_l = []
bb_pct_l = []
k_bb_pct_l = []
avg_l = []
whip_l = []
babip_l = []
lob_pct_l = []
fip_l = []
xfip_l = []
k_9_r = []
bb_9_r = []
k_bb_r = []
hr_9_r = []
k_pct_r = []
bb_pct_r = []
k_bb_pct_r = []
avg_r = []
whip_r = []
babip_r = []
lob_pct_r = []
fip_r = []
xfip_r = []
# Engineering features
k_9_l_tot = []
bb_9_l_tot = []
k_bb_l_tot = []
hr_9_l_tot = []
k_pct_l_tot = []
bb_pct_l_tot = []
k_bb_pct_l_tot = []
avg_l_tot = []
whip_l_tot = []
babip_l_tot = []
lob_pct_l_tot = []
fip_l_tot = []
xfip_l_tot = []
k_9_r_tot = []
bb_9_r_tot = []
k_bb_r_tot = []
hr_9_r_tot = []
k_pct_r_tot = []
bb_pct_r_tot = []
k_bb_pct_r_tot = []
avg_r_tot = []
whip_r_tot = []
babip_r_tot = []
lob_pct_r_tot = []
fip_r_tot = []
xfip_r_tot = []
for i, row in pitchers.iterrows():
    # Pulling stats of pitcher against lefty hitters
    pitcher_df_l = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'L')&(pitch_df['Name']==row['Name']),:]
    if len(pitcher_df_l) == 0:
        for col in pitcher_df_l.columns:
            pitcher_df_l[col] = [float('NaN')]
    k_9_l.append(pitcher_df_l['K/9'].values[0])
    bb_9_l.append(pitcher_df_l['BB/9'].values[0])
    k_bb_l.append(pitcher_df_l['K/BB'].values[0])
    hr_9_l.append(pitcher_df_l['HR/9'].values[0])
    k_pct_l.append(pitcher_df_l['K%'].values[0])
    bb_pct_l.append(pitcher_df_l['BB%'].values[0])
    k_bb_pct_l.append(pitcher_df_l['K-BB%'].values[0])
    avg_l.append(pitcher_df_l['AVG'].values[0])
    whip_l.append(pitcher_df_l['WHIP'].values[0])
    babip_l.append(pitcher_df_l['BABIP'].values[0])
    lob_pct_l.append(pitcher_df_l['LOB%'].values[0])
    fip_l.append(pitcher_df_l['FIP'].values[0])
    xfip_l.append(pitcher_df_l['xFIP'].values[0])
    # Feature Engineering
    k_9_l_tot.append(pitcher_df_l['K/9'].values[0] * row['num_L_bats'])
    bb_9_l_tot.append(pitcher_df_l['BB/9'].values[0] * row['num_L_bats'])
    k_bb_l_tot.append(pitcher_df_l['K/BB'].values[0] * row['num_L_bats'])
    hr_9_l_tot.append(pitcher_df_l['HR/9'].values[0] * row['num_L_bats'])
    k_pct_l_tot.append(pitcher_df_l['K%'].values[0] * row['num_L_bats'])
    bb_pct_l_tot.append(pitcher_df_l['BB%'].values[0] * row['num_L_bats'])
    k_bb_pct_l_tot.append(pitcher_df_l['K-BB%'].values[0] * row['num_L_bats'])
    avg_l_tot.append(pitcher_df_l['AVG'].values[0] * row['num_L_bats'])
    whip_l_tot.append(pitcher_df_l['WHIP'].values[0] * row['num_L_bats'])
    babip_l_tot.append(pitcher_df_l['BABIP'].values[0] * row['num_L_bats'])
    lob_pct_l_tot.append(pitcher_df_l['LOB%'].values[0] * row['num_L_bats'])
    fip_l_tot.append(pitcher_df_l['FIP'].values[0] * row['num_L_bats'])
    xfip_l_tot.append(pitcher_df_l['xFIP'].values[0] * row['num_L_bats'])
    # Pulling stats against righty hitters
    pitcher_df_r = pitch_df.sort_values(by = 'IP', ascending = False).loc[(pitch_df['batter']== 'R')&(pitch_df['Name']== row['Name']),:]
    if len(pitcher_df_r) == 0:
        for col in pitcher_df_r.columns:
            pitcher_df_r[col] = [float('NaN')]
    k_9_r.append(pitcher_df_r['K/9'].values[0])
    bb_9_r.append(pitcher_df_r['BB/9'].values[0])
    k_bb_r.append(pitcher_df_r['K/BB'].values[0])
    hr_9_r.append(pitcher_df_r['HR/9'].values[0])
    k_pct_r.append(pitcher_df_r['K%'].values[0])
    bb_pct_r.append(pitcher_df_r['BB%'].values[0])
    k_bb_pct_r.append(pitcher_df_r['K-BB%'].values[0])
    avg_r.append(pitcher_df_r['AVG'].values[0])
    whip_r.append(pitcher_df_r['WHIP'].values[0])
    babip_r.append(pitcher_df_r['BABIP'].values[0])
    lob_pct_r.append(pitcher_df_r['LOB%'].values[0])
    fip_r.append(pitcher_df_r['FIP'].values[0])
    xfip_r.append(pitcher_df_r['xFIP'].values[0])
    # Feature Engineering
    k_9_r_tot.append(pitcher_df_r['K/9'].values[0] * row['num_R_bats'])
    bb_9_r_tot.append(pitcher_df_r['BB/9'].values[0] * row['num_R_bats'])
    k_bb_r_tot.append(pitcher_df_r['K/BB'].values[0] * row['num_R_bats'])
    hr_9_r_tot.append(pitcher_df_r['HR/9'].values[0] * row['num_R_bats'])
    k_pct_r_tot.append(pitcher_df_r['K%'].values[0] * row['num_R_bats'])
    bb_pct_r_tot.append(pitcher_df_r['BB%'].values[0] * row['num_R_bats'])
    k_bb_pct_r_tot.append(pitcher_df_r['K-BB%'].values[0] * row['num_R_bats'])
    avg_r_tot.append(pitcher_df_r['AVG'].values[0] * row['num_R_bats'])
    whip_r_tot.append(pitcher_df_r['WHIP'].values[0] * row['num_R_bats'])
    babip_r_tot.append(pitcher_df_r['BABIP'].values[0] * row['num_R_bats'])
    lob_pct_r_tot.append(pitcher_df_r['LOB%'].values[0] * row['num_R_bats'])
    fip_r_tot.append(pitcher_df_r['FIP'].values[0] * row['num_R_bats'])
    xfip_r_tot.append(pitcher_df_r['xFIP'].values[0] * row['num_R_bats'])
pitchers['k_9_l'] = k_9_l
pitchers['bb_9_l'] = bb_9_l
pitchers['k_bb_l'] = k_bb_l
pitchers['hr_9_l'] = hr_9_l
pitchers['k_pct_l'] = k_pct_l
pitchers['bb_pct_l'] = bb_pct_l
pitchers['k_bb_pct_l'] = k_bb_pct_l
pitchers['bat_avg_l'] = avg_l
pitchers['whip_l'] = whip_l
pitchers['babip_l'] = babip_l
pitchers['lob_pct_l'] = lob_pct_l
pitchers['fip_l'] = fip_l
pitchers['xfip_l'] = xfip_l
# Right handed batter stats
pitchers['k_9_r'] = k_9_r
pitchers['bb_9_r'] = bb_9_r
pitchers['k_bb_r'] = k_bb_r
pitchers['hr_9_r'] = hr_9_r
pitchers['k_pct_r'] = k_pct_r
pitchers['bb_pct_r'] = bb_pct_r
pitchers['k_bb_pct_r'] = k_bb_pct_r
pitchers['bat_avg_r'] = avg_r
pitchers['whip_r'] = whip_r
pitchers['babip_r'] = babip_r
pitchers['lob_pct_r'] = lob_pct_r
pitchers['fip_r'] = fip_r
pitchers['xfip_r'] = xfip_r
# feature engineered left handed batter stats
pitchers['k_9_l_tot'] = k_9_l_tot
pitchers['bb_9_l_tot'] = bb_9_l_tot
pitchers['k_bb_l_tot'] = k_bb_l_tot
pitchers['hr_9_l_tot'] = hr_9_l_tot
pitchers['k_pct_l_tot'] = k_pct_l_tot
pitchers['bb_pct_l_tot'] = bb_pct_l_tot
pitchers['k_bb_pct_l_tot'] = k_bb_pct_l_tot
pitchers['bat_avg_l_tot'] = avg_l_tot
pitchers['whip_l_tot'] = whip_l_tot
pitchers['babip_l_tot'] = babip_l_tot
pitchers['lob_pct_l_tot'] = lob_pct_l_tot
pitchers['fip_l_tot'] = fip_l_tot
pitchers['xfip_l_tot'] = xfip_l_tot
# feature engineered righ handed batter stats
pitchers['k_9_r_tot'] = k_9_r_tot
pitchers['bb_9_r_tot'] = bb_9_r_tot
pitchers['k_bb_r_tot'] = k_bb_r_tot
pitchers['hr_9_r_tot'] = hr_9_r_tot
pitchers['k_pct_r_tot'] = k_pct_r_tot
pitchers['bb_pct_r_tot'] = bb_pct_r_tot
pitchers['k_bb_pct_r_tot'] = k_bb_pct_r_tot
pitchers['bat_avg_r_tot'] = avg_r_tot
pitchers['whip_r_tot'] = whip_r_tot
pitchers['babip_r_tot'] = babip_r_tot
pitchers['lob_pct_r_tot'] = lob_pct_r_tot
pitchers['fip_r_tot'] = fip_r_tot
pitchers['xfip_r_tot'] = xfip_r_tot

In [56]:
pitchers

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,hr_9_r_tot,k_pct_r_tot,bb_pct_r_tot,k_bb_pct_r_tot,bat_avg_r_tot,whip_r_tot,babip_r_tot,lob_pct_r_tot,fip_r_tot,xfip_r_tot
9,BAL,6/17/2021,1,605347.0,Jorge Lopez,SP,Y,NaN,3,6,...,8.470588,1.179039,0.497817,0.681223,1.782178,9.294118,2.052632,4.120879,29.067694,25.595993
19,CLE,6/17/2021,1,669212.0,Eli Morgan,SP,Y,NaN,3,6,...,0.000000,0.400000,0.800000,-0.400000,3.000000,18.000000,3.000000,3.000000,27.825475,43.535194
28,ARI,6/17/2021,1,668678.0,Zac Gallen,SP,Y,NaN,5,3,...,3.521739,1.041667,0.263889,0.777778,0.538462,3.018634,0.736842,2.588496,9.582004,9.741618
37,SF,6/17/2021,1,592332.0,Kevin Gausman,SP,N,NaN,5,3,...,2.806931,0.938931,0.206107,0.732824,0.520661,2.673267,0.677419,2.270916,8.635769,9.279301
57,TOR,6/17/2021,1,643615.0,TJ Zeuch,SP,N,NaN,2,7,...,14.264151,0.788732,0.492958,0.295775,1.484848,7.528302,1.296296,5.223881,42.369710,34.577205
66,CHC,6/17/2021,1,543294.0,Kyle Hendricks,SP,N,NaN,4,4,...,5.811659,0.616393,0.118033,0.498361,1.068493,4.681614,1.123404,3.157895,17.771554,16.172770
75,NYM,6/17/2021,1,573186.0,Marcus Stroman,SP,N,NaN,4,4,...,4.909091,0.845528,0.260163,0.585366,0.904348,4.636364,1.035294,2.957746,15.823044,14.920812
84,STL,6/17/2021,1,607231.0,John Gant,SP,N,NaN,3,5,...,0.000000,1.024845,0.559006,0.465839,1.214286,7.027027,1.559633,4.245283,14.546862,18.842125
93,ATL,6/17/2021,1,450203.0,Charlie Morton,SP,N,NaN,3,5,...,2.934783,1.161049,0.280899,0.880150,1.361789,6.684783,1.730769,3.420398,14.560169,16.595528
103,CHW,6/17/2021,1,656302.0,Dylan Cease,SP,N,NaN,2,7,...,8.647059,1.757848,0.910314,0.847534,1.694737,10.294118,2.100000,5.392749,31.618390,34.570444


**Data Description:** This worksheet provides a framework for creating dataframes for each days' players and stats. I will combine these dataframes across many days to get my final training data. These data frames have all of the stats that I could potentially be interested in, the features will be reduced for modeling as there is a significant amount of correlation between many of the features. The batters dataframe includes the batters stats against pitchers who throw with the same hand as the opposing pitcher, and includes the stats of how the opposing pitcher performs against batters with the same handedness of the batter. The pitchers dataframe contains each pitcher and their stats against both right handed and left handed batters. It also contains the number of right and left handed batters in the opposing lineup. I engineered features that are the pitchers stats vs the handedness of a batter multiplied by the number of those handed batters in the lineup. My final feature is is_nl. This is 1 if the lineup they are facing includes a pitcher and 0 if not.

**Data Dictionary:**   
team code: The team the player plays for.  
Name: The players name  
Batting order: The order the batters hit in the lineup. Is SP if player is a pitcher.  
opp_pitch_hand(batters): The hand that the opposing pitcher throws with  
opp_pitch(batters):The name of the opposing pitcher. 
opp_x(batters): This is the format for column names of stats that the opposing pitcher has against batters of the same handedness of the batter.  
(batters): There are also many columns of stats for how the batter performs against pitchers with the same handedness of the opposing pitcher.    
num_L_bats(pitchers): The number of left handed batters in the opposing lineup.  
num_R_bats(pitchers): The number of right handed batters in the opposing lineup.  
x_(l/r) (pitchers): stats describing how the pitcher performs against either left or right handed batters.    
x_(l/r)_tot (pitchers): The stats of how a pitcher performs against both hands multiplied by the number of those handed batters in the lineup.

In [57]:
scores = pd.read_csv('../datasets/6_17_target.csv', delimiter = ';')
scores.head()

,Date,GID,MLB_ID,Name,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,20210617,1575,450203,"Morton, Charlie",1.0,9,1.0,32.25,7200.0,ATL,v stl,NaN,4,0
1,20210617,125a,543294,"Hendricks, Kyle",1.0,9,1.0,29.10,8800.0,CHC,@ nym,NaN,2,0
2,20210617,116k,592332,"Gausman, Kevin",1.0,9,1.0,27.00,9600.0,SFO,v ari,NaN,10,3
3,20210617,123u,573186,"Stroman, Marcus",1.0,9,1.0,24.75,8600.0,NYM,v chc,NaN,0,2
4,20210617,154y,664353,"Urquidy, Jose",1.0,0,1.0,22.75,7900.0,HOU,v chw,NaN,10,2


In [58]:
def clean_names(name):
    name = name.replace(',','')
    name = name.replace('.','')
    name = name.replace('-hitter','')
    name = name.replace('-pitcher','')
    names = name.split()
    first = names.pop(-1)
    names.insert(0, first)
    name = ' '.join(names)
    name = name.replace('James Happ', 'JA Happ')
    return name

In [59]:
scores['Name'] = [clean_names(name) for name in scores['Name']]

In [60]:
pitch_s = scores[scores['DK posn'] == 1]
hit_s = scores[scores['DK posn'] != 1]

In [61]:
pitch_final = pd.merge(left = pitchers, right = pitch_s, how = 'inner')

In [62]:
bat_final = pd.merge(left = batters, right = hit_s,
         how = 'inner').dropna(how = 'any', subset = ['bats', 'DK pts', 'Starter'])

In [63]:
bat_final = bat_final[bat_final['dblhdr'].isna()]

In [64]:
bat_final.to_csv('../merged_data/hitters_6_17.csv')
pitch_final.to_csv('../merged_data/pitchers_6_17.csv')

In [65]:
bat_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,opp_pitch_hand,opp_pitch,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,BAL,6/17/2021,1,656775.0,Cedric Mullins,1,Y,CF,R,Eli Morgan,...,1.0,1,7.0,2.0,3700.0,BAL,@ cle,NaN,3,10
1,BAL,6/17/2021,1,641820.0,Trey Mancini,2,Y,1B,R,Eli Morgan,...,1.0,2,3.0,8.0,4800.0,BAL,@ cle,NaN,3,10
2,BAL,6/17/2021,1,520471.0,Freddy Galvis,3,Y,SS,R,Eli Morgan,...,1.0,3,6.0,2.0,3300.0,BAL,@ cle,NaN,3,10
3,BAL,6/17/2021,1,663624.0,Ryan Mountcastle,4,Y,DH,R,Eli Morgan,...,1.0,4,37.0,6.0,3100.0,BAL,@ cle,NaN,3,10
4,BAL,6/17/2021,1,623993.0,Anthony Santander,5,Y,RF,R,Eli Morgan,...,1.0,5,7.0,3.0,3000.0,BAL,@ cle,NaN,3,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113,TB,6/17/2021,1,621433.0,Brett Phillips,9,N,NaN,R,Justin Dunn,...,1.0,9,7.0,14.0,2300.0,TAM,@ sea,NaN,5,6
114,SEA,6/17/2021,1,641487.0,JP Crawford,1,N,NaN,L,Rich Hill,...,1.0,1,6.0,5.0,3400.0,SEA,v tam,NaN,6,5
115,SEA,6/17/2021,1,571745.0,Mitch Haniger,2,N,NaN,L,Rich Hill,...,1.0,2,7.0,5.0,4800.0,SEA,v tam,NaN,6,5
117,SEA,6/17/2021,1,664034.0,Ty France,4,N,NaN,L,Rich Hill,...,1.0,3,3.0,16.0,2700.0,SEA,v tam,NaN,6,5


In [66]:
pitch_final

,team code,game_date,game_number,mlb id,Name,batting order,confirmed,position,num_L_bats,num_R_bats,...,Starter,Bat order,DK posn,DK pts,DK sal,Team,Oppt,dblhdr,Tm Runs,Opp Runs
0,BAL,6/17/2021,1,605347.0,Jorge Lopez,SP,Y,NaN,3,6,...,1.0,0,1.0,0.95,7300.0,BAL,@ cle,NaN,3,10
1,CLE,6/17/2021,1,669212.0,Eli Morgan,SP,Y,NaN,3,6,...,1.0,0,1.0,8.65,7000.0,CLE,v bal,NaN,10,3
2,ARI,6/17/2021,1,668678.0,Zac Gallen,SP,Y,NaN,5,3,...,1.0,9,1.0,0.40,8700.0,ARI,@ sfo,NaN,3,10
3,SF,6/17/2021,1,592332.0,Kevin Gausman,SP,N,NaN,5,3,...,1.0,9,1.0,27.00,9600.0,SFO,v ari,NaN,10,3
4,TOR,6/17/2021,1,643615.0,TJ Zeuch,SP,N,NaN,2,7,...,1.0,0,1.0,2.55,6200.0,TOR,v nyy,NaN,4,8
5,CHC,6/17/2021,1,543294.0,Kyle Hendricks,SP,N,NaN,4,4,...,1.0,9,1.0,29.10,8800.0,CHC,@ nym,NaN,2,0
6,NYM,6/17/2021,1,573186.0,Marcus Stroman,SP,N,NaN,4,4,...,1.0,9,1.0,24.75,8600.0,NYM,v chc,NaN,0,2
7,STL,6/17/2021,1,607231.0,John Gant,SP,N,NaN,3,5,...,1.0,9,1.0,10.40,7500.0,STL,@ atl,NaN,0,4
8,ATL,6/17/2021,1,450203.0,Charlie Morton,SP,N,NaN,3,5,...,1.0,9,1.0,32.25,7200.0,ATL,v stl,NaN,4,0
9,CHW,6/17/2021,1,656302.0,Dylan Cease,SP,N,NaN,2,7,...,1.0,0,1.0,-0.70,9500.0,CHW,@ hou,NaN,2,10
